In [4]:
# there is a issue on panda after 1.0.0, need 0.25.3
import pandas as pd
pd.__version__

'0.25.3'

In [7]:
review_json_path = 'yelp_academic_dataset_review.json'

import pandas as pd

# Set chunk size (smaller if dataset is smaller)
# 2020 Yelp review.json has more than 8 million reviews(rows)
#1 million per time in this dataset will cost more than 6GB of my RAM
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size, encoding="utf8")


In [10]:
#Merge Review.json and Business.json file
## There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool','user_id','business_id'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Show feedback on progress
    print(f"{chunk.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

ValueError: I/O operation on closed file.

In [11]:
df=df.drop(['date'], axis=1)

In [12]:
df.sample(3)

,review_stars,text
1992079,2,I ordered through them on Uber eats and asked ...
3234990,2,I have to add to Andrew's review.....\n\nI jus...
921106,5,I was finally able to understand the issues of...


In [13]:
#Review Filter:
#drop review that length are less than 50
df['text_length'] = df['text'].str.len()

In [14]:
df

,review_stars,text,text_length
0,4,What a treat it was to find Sidebar in Downtow...,622
1,5,I love Sunchain! I have been a customer for ov...,968
2,3,The food is good. But the service is suck comp...,361
3,1,We went here the first week they opened and we...,1393
4,1,"Beware, discriminating amongst the gay communi...",179
...,...,...,...
7021117,1,"Fricken unbelievable, I ordered 2 space heater...",770
7021118,3,Solid American food with a southern comfort fl...,1444
7021119,5,I'm honestly not sure how I have never been to...,1225
7021120,3,Food was decent but I will say the service too...,327


In [15]:
df=df.drop(df[df.text_length < 50].index)
df

,review_stars,text,text_length
0,4,What a treat it was to find Sidebar in Downtow...,622
1,5,I love Sunchain! I have been a customer for ov...,968
2,3,The food is good. But the service is suck comp...,361
3,1,We went here the first week they opened and we...,1393
4,1,"Beware, discriminating amongst the gay communi...",179
...,...,...,...
7021117,1,"Fricken unbelievable, I ordered 2 space heater...",770
7021118,3,Solid American food with a southern comfort fl...,1444
7021119,5,I'm honestly not sure how I have never been to...,1225
7021120,3,Food was decent but I will say the service too...,327


In [16]:
df=df.drop(['text_length'], axis=1)
df

,review_stars,text
0,4,What a treat it was to find Sidebar in Downtow...
1,5,I love Sunchain! I have been a customer for ov...
2,3,The food is good. But the service is suck comp...
3,1,We went here the first week they opened and we...
4,1,"Beware, discriminating amongst the gay communi..."
...,...,...
7021117,1,"Fricken unbelievable, I ordered 2 space heater..."
7021118,3,Solid American food with a southern comfort fl...
7021119,5,I'm honestly not sure how I have never been to...
7021120,3,Food was decent but I will say the service too...


In [17]:
df.to_csv("yelp_reviews.csv", index=False)